# **Création d'un modèle de reconnaissance vocale**

Le code ci-dessous a pour objectif d'implémenter un modèle de reconnaissance vocale sur des plaques d'immatriculation à 7 caractères. Il utilise notamment la bibliothèque tensorflow.

Dans un premier temps, on récupère la base de données d'apprentissage et on en récupère les descripteurs que l'on fournira en entrée à notre modèle.
Puis dans un second temps, on crée le modèle et on effectue son entrainement.

Le modèle est un réseau de neurones, constitué de couches de convolution et de couches récurrentes (LSTM).

## Importations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

import librosa
import os
import pickle

import IPython.display as ipd

from random import randint

 # **I) Récupération des données et extraction des mfcc**

## **1) Récupération des données**

Les données sont stockées sur Google Drive car cela est plus pratique afin de travailler avec Google colab. On va donc charger le drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_PATH = '/content/drive/MyDrive/Projet speechbrain/'

Mounted at /content/drive


In [ ]:
# On récupère les fichiers audio

def get_wav_files(path):
  """
  get_wav_files renvoie la liste des fichiers audio, et la liste des labels correspondant
  -------------
  Arguments
  ---------
  path: str
    Le chemin du dossier qui contient les fichiers audio
  
  Output
  ------
  waves, y: float list list, str list
    La liste des fichiers audio, et la liste des labels (plaques d'immatriculation)
  """
    
  paths = [path + wavfile for wavfile in os.listdir(path)]
  waves = []
  y = []
  for path in paths:
    wave, sr = librosa.load(path, mono=True, sr=None)
    waves.append(wave)
    y.append(path[:-4].split('/')[-1])

  waves = np.asarray(waves)

  return waves, y

waves, y = get_wav_files(path = DRIVE_PATH + "Audio/")

waves_test, y_test = get_wav_files(path = DRIVE_PATH + 'Audio test/')

In [ ]:
def modify_wav(wave, ratio=50):
  """
  modify_wav prend en entrée un fichier audio, et y ajoute du bruit (que l'on contrôle avec le paramètre ratio)
  -------------
  Arguments
  ---------
  wave: float list
    Un fichier audio
    
  ratio: float
    L'augmentation de données est faite par l'ajout d'un bruit gaussien au signal
    La loi normale utilisée est une loi d'espérance 0 et de variance max/ratio, où max est la valeur maximale de la liste représentant un fichier audio
    
  Output
  ------
  new_wave: float list
    Fichier audio en entrée avec du bruit
  """

  maxi = np.max(wave)
  shape = wave.shape

  noise = np.random.normal(0, maxi/ratio, shape)

  new_wave = wave + noise
  return new_wave

def modify_list_wav(waves, ratio=50):
  """
  modify_wav prend en entrée une liste de fichiers audio, et y ajoute du bruit
  -------------
  Arguments
  ---------
  waves: float list list
    Une liste de fichier audio fichier audio
    
  ratio: float
    L'augmentation de données est faite par l'ajout d'un bruit gaussien au signal
    La loi normale utilisée est une loi d'espérance 0 et de variance max/ratio, où max est la valeur maximale de la liste représentant un fichier audio
    
  Output
  ------
  new_wave: float list list
    Fichier audio en entrée avec du bruit
  """
  new = []
  for wave in waves:
    new.append(modify_wav(wave=wave, ratio=ratio))

  return np.asarray(new)

wave, sr = librosa.load(DRIVE_PATH + "Audio/ag-273-qs.wav")

# Test de l'ajout de bruit
test = modify_wav(wave, ratio=30)
ipd.Audio(test, rate=sr)

## **2) Extraction des mfcc**

In [ ]:
# On extrait les mfcc des fichiers audio

def wav2mfcc(wave, n_mfcc=40):
    """
    wav2mfcc renvoie la liste des mfcc à partir d'un fichier audio
    -------------
    Arguments
    ---------
    wave: float list
      Un fichier audio
    
    n_mfcc: int
      Le nombre de mfcc que l'on extrait
    
    Output
    ------
    mfcc: float list list
      Liste des mfcc
    """

    mfcc = librosa.feature.mfcc(wave, n_mfcc=n_mfcc)
    
    return mfcc


def get_mfccs(waves, labels, n_mfcc=40, data_augmentation=0, ratio=50, display_steps=False):
    """
    get_mfccs renvoie la liste des mfccs du jeu de données, en permettant de la data augmentation
    -------------
    Arguments
    ---------
    waves: float list list
      Un fichier audio
    
    labels: str list
      Liste des labels (plaques d'immatriculation)

    n_mfcc: int
      Le nombre de mfcc que l'on extrait

    data_augmentation: int
      Entier qui donne le nombre de fois par lequel on multiplie la quantité de notre jeu de données
      Si data_augmentation=1 et que le dataset contient 1000 fichiers audio, la sortie contiendra 2000 fichiers audio
      Pour ne pas avoir d'augmentation de données, mettre data_augmentation=0

    ratio: float
      L'augmentation de données est faite par l'ajout d'un bruit gaussien au signal
      La loi normale utilisée est une loi d'espérance 0 et de variance max/ratio, où max est la valeur maximale de la liste représentant un fichier audio
    
    display_steps: bool
      Vaut True si on veut afficher l'étape à laquelle l'algo est, et False sinon
      
    Output
    ------
    X, y: float list list list, str list
      Liste des mfcc pour tout le dataset, et liste des labels
    """

    X = []
    y = []
    n = len(waves)
    wave_shape = waves.shape[1:]
    for i in range(n):
        if (display_steps):
          print("----- Step {}/{} -----".format(i, n))
        wave = waves[i]
        X.append(wav2mfcc(wave, n_mfcc=n_mfcc))
        y.append(labels[i])
        for j in range(data_augmentation):
            maxi = np.max(wave)
            X.append(wav2mfcc(modify_wav(wave, ratio=ratio), n_mfcc=n_mfcc))
            y.append(labels[i])

    X = np.asarray(X)
    X = np.expand_dims(X, axis=3)

    return X, y

n_mfcc = 40

X, y = get_mfccs(waves= waves, labels=y, n_mfcc=n_mfcc, data_augmentation=10, ratio=30, display_steps=True)

X_test, y_test = get_mfccs(waves=waves_test, labels=y_test, n_mfcc=n_mfcc, data_augmentation=0, display_steps=False)

## **3) Encodage des labels (plaques d'immatriculation)**

In [ ]:
# On récupère les données nécessaires pour l'encodage des labels

def get_carac_data():
    """
    get_carac_data renvoie des dictionnaires de la forme {'a': 0, 'b': 1} pour encoder les caractères
    -----------
    Output
    ------
    letter_to_label: dict
      Dictionnaire de la forme {'a': 0, 'b': 1} pour l'encodage des caractères

    label_to_letter: dict
      Dictionnaire inversé de letter_to_label

    num_carac: int
      Le nombre de caractères différents
    """

    letter_to_label = {}
    vocab = 'abcdefghijklmnopqrstuvwxyz0123456789'

    num_carac = len(vocab)
    n = len(vocab)
    for i in range(n):
        letter_to_label[vocab[i]] = i

    label_to_letter = {v: k for k, v in letter_to_label.items()}
    
    return letter_to_label, label_to_letter, num_carac

letter_to_label, label_to_letter, num_carac = get_carac_data()

In [ ]:
# On encode les labels 

def change_labels(y):
    """
    change_labels transforme chaque plaque de la forme 'bf-467-zg' en une liste de la forme ['b', 'f', '4', '6', '7', 'z', 'g']
    -------------
    Arguments
    ---------
    y: str list
      La liste des labels (plaques d'immatriculation), 
      de la forme ['bf-467-zg', 'fd-785-gt', ...]
    
    Output
    ------
    new: str list list
      Une liste de la forme [['b', 'f', '4', '6', '7', 'z', 'g'],
                             ['f', 'd', '7', '8', '5', 'g', 't' ]]
    """

    n = len(y)
    new = []
    for i in range(n):
        line = []
        for j in range(9):
            if j != 2 and j != 6:
                line.append(y[i][j])
        new.append(line)
        
    return new

def encode_labels(y):
    """
    encode_labels prend en entrée une liste de plaques de la forme ['b', 'f', '4', '6', '7', 'z', 'g'], et renvoie la liste des encodages de la forme [1, 5, 31, 33, 34, 25, 12]
    -------------
    Arguments
    ---------
    y: str list list
      Les labels de la forme [['b', 'f', '4', '6', '7', 'z', 'g'],
                              ['f', 'd', '7', '8', '5', 'g', 't' ]]
    
    Output
    ------
    y: int list list
      Liste de la forme [[4, 17, 18, 20, 15, 31, 20],
                         [6, 18, 20, 5, 6, 34, 8]]
      qui encode les caractères
    """

    n = len(y)
    for i in range(n):
        m = len(y[i])
        for j in range(m):
            y[i][j] = letter_to_label[y[i][j]]
    
    return y

def one_hot_encoding(y):
    """
    one_hot_encoding prend en entrée une liste de plaques encodées par des entiers (de la forme [[4, 17, 18, 20, 15, 31, 20], [6, 18, 20, 5, 6, 34, 8]]), et renvoie une version avec un one hot encoding
    -------------
    Arguments
    ---------
    y: int list list
      Les labels de la forme [[4, 17, 18, 20, 15, 31, 20],
                              [6, 18, 20, 5, 6, 34, 8]]
    
    Output
    ------
    y: int list list
      Liste de la forme [[0,0,0,1,0,0,0,0,0,0,0,0...],
                         [0,0,0,0,0,1,0,0,0,0,0,0...]]
      
      Explication: par exemple, dans la liste [4, 17, 18, 20, 15, 31, 20],
      on transforme le 4 en [0,0,0,1,0,0...], puis le 17 de la même manière, etc, puis on concatène toutes ces listes
      On obtient alors le nouveau label

      Et on fait cela pour tous les labels
    """
   
    res = []
    n = len(y)
    for i in range(n):
        new = []
        for carac in y[i]:
            new_line = [0 for _ in range(num_carac)]
            new_line[carac] = 1
            new.append(new_line)
        res.append(new)
    
    res = np.asarray(res)
    res = res.reshape(-1, 7*36)
    return res

def get_labels(y):
  """
  get_labels utilise toutes les fonctions précédentes, et transforme une liste des labels sous forme de plaques, 
  et les renvoie avec un one hot encoding
  -------------
  Arguments
  ---------
  y: str list
    Liste de la forme ['bd-459-gd', 'gt-486-ht']
    
  Output
  ------
  y: int list list
    Les labels de la forme [[0,0,0,1,0,0,0,0,0,0,0,0...],
                            [0,0,0,0,0,1,0,0,0,0,0,0...]]
  """    

  y = change_labels(y)
  y = encode_labels(y)
  y = one_hot_encoding(y)
  return y


y = get_labels(y)
y_test = get_labels(y_test)

In [ ]:
# On définit une fonction qui servira à passer d'un encodage one-hot à la plaque d'immatriculation

def translate(y):
    """
    translate prend en entrée la liste des labels sous forme one hot encoding, et renvoie les plaques d'immatriculation correspondantes
    -------------
    Arguments
    ---------
    y: int list list
      Les labels de la forme [[0,0,0,1,0,0,0,0,0,0,0,0...],
                              [0,0,0,0,0,1,0,0,0,0,0,0...]]
    
    Output
    ------
    y: str list
      Liste de la forme ['bd-459-gd', 'gt-486-ht']
    """    
    res = []
    n = len(y)
    y = np.asarray(y)
    y = y.reshape(-1, 7, 36)

    for i in range(n):
        plaque = ""
        for l in y[i]:
          label = np.argmax(l)
          plaque += label_to_letter[label]

        plaque = plaque[:2] + "-" + plaque[2:5] + "-" + plaque[5:]
        res.append(plaque)

    return res

## **4) Sauvegarde des données**

In [ ]:
def save(path='Savings/'):
  """Sauvegarde les données"""
  with open(path + 'mfccs.pickle', 'wb') as handle:
      pickle.dump(X, handle, protocol=pickle.HIGHEST_PROTOCOL)

  with open(path + 'mfccs_test.pickle', 'wb') as handle:
      pickle.dump(X_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

  with open(path + 'labels.pickle', 'wb') as handle:
      pickle.dump(y, handle, protocol=pickle.HIGHEST_PROTOCOL)

  with open(path + 'labels_test.pickle', 'wb') as handle:
      pickle.dump(y_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

# save(path=DRIVE_PATH + 'Savings/')

# **II) Développement du modèle**

## **1) Chargement des données**

In [ ]:
from sklearn.utils import shuffle

def load_data(path):
  with open(path+'mfccs.pickle', 'rb') as handle:
      X = pickle.load(handle)
      
  with open(path+'labels.pickle', 'rb') as handle:
      y = pickle.load(handle)

  with open(path+'mfccs_test.pickle', 'rb') as handle:
      X_test = pickle.load(handle)
      
  with open(path+'labels_test.pickle', 'rb') as handle:
      y_test = pickle.load(handle)
  
  return X, X_test, y, y_test

X, X_test, y, y_test = load_data(path=DRIVE_PATH + 'Savings/')

# On mélange les données pour l'entrainement du modèle
X, y = shuffle(X, y, random_state=0)

## **2) Création du modèle**

Le modèle prend en entrée la liste des mfcc.

Cette liste passe d'abord part des couches de convolution + pooling. (ce qui agit donc comme wav2vec)  
Ensuite, on passe par un encodeur constitué de cellules LSTM.  
Enfin, on passe par un décodeur constitué de cellules LSTM + une couche dense

### **a) Couches de convolution**

In [ ]:
class ConvLayers(tf.keras.layers.Layer):
    """
    Ensemble des couches de convolution + pooling
    """
    def __init__(self, **kwargs):
        super(**kwargs).__init__()
        
    def build(self, input_shape):
        self.pooling = layers.AveragePooling2D((2,2))
        self.conv1 = layers.Conv2D(filters=128, kernel_size=(3,5), activation='relu', padding='same')
        self.conv2 = layers.Conv2D(filters=64, kernel_size=(3,5), activation='relu', padding='same')
        self.conv3 = layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same')
        self.dense = layers.Dense(1)
        self.reshape = layers.Reshape((-1, 23))
        super().build(input_shape)
        
    def call(self, x):
        x = self.conv1(x)
        x = self.pooling(x)
        
        x = self.conv2(x)
        x = self.pooling(x)
        
        x = self.conv3(x)
        x = self.pooling(x)
        
        x = self.dense(x)

        x = self.reshape(x)

        return x

def test():
    layer_input = tf.keras.Input(shape=X.shape[1:])
    conv_output = ConvLayers()(layer_input)
    
    model = tf.keras.Model(layer_input, conv_output)
    model.summary()
    
# test()

### **b) Encodeur**

In [ ]:
class Encoder(tf.keras.layers.Layer):
    """
    Encodeur constitué de cellules LSTM
    """
    def __init__(self, nb_units, dropout, **kwargs):
        self.nb_units = nb_units
        self.dropout = dropout
        super(**kwargs).__init__()
    
    def build(self, input_shape):
        self.lstm = layers.LSTM(self.nb_units, return_sequences=False, dropout=self.dropout, recurrent_dropout=self.dropout)

        super().build(input_shape)
        
    def call(self, x):
        x = self.lstm(x)
        
        return x
    
def test():
    layer_input = tf.keras.Input(shape=(40, 188, 1))
    conv_output = ConvLayers()(layer_input)
    encoder_output = Encoder(nb_units=7*32, dropout=0.5)(conv_output)
    
    model = tf.keras.Model(layer_input, encoder_output)
    model.summary()
    
# test()

In [ ]:
# La couche ci-dessous sert à implémenter des mécanismes d'attention. Nous ne l'avons néanmoins pas utilisée dans notre modèle pour des soucis techniques.

class Attention(tf.keras.layers.Layer):
    """
    Couche d'attention
    """
    def __init__(self, nb_units, **kwargs):
        self.nb_units = nb_units
        super(**kwargs).__init__()
    
    def build(self, input_shape):
        self.query_layer = layers.Dense(32)
        self.value_layer = layers.Dense(32)
        self.key_layer = layers.Dense(32)

        super().build(input_shape)
        
    def call(self, x):
        Q = self.query_layer(x)
        K = self.key_layer(x)
        V = self.value_layer(x)
        QK = tf.matmul(Q, K, transpose_b=True)
        QK = QK / tf.math.sqrt(256.)
        softmax_QK = tf.nn.softmax(QK, axis=-1)
        attention = tf.matmul(softmax_QK, V)

        return attention

            
def test():
    layer_input = tf.keras.Input(shape=(40, 188, 1))
    conv_output = ConvLayers()(layer_input)
    encoder_output = Encoder(nb_units=7*32, dropout=0.5)(conv_output)
    
    model = tf.keras.Model(layer_input, encoder_output)
    model.summary()
    
# test()

### **c) Décodeur**

In [ ]:
class DecoderRNN(tf.keras.layers.Layer):
    """
    Ensemble des cellules LSTM du décodeur
    """
    def __init__(self, nb_units, dropout, **kwargs):
        self.nb_units = nb_units
        self.dropout = dropout
        super(**kwargs).__init__()
    
    def build(self, input_shape):
        self.flatten = layers.Flatten()
        self.reshape = layers.Reshape((7,64))
        self.LSTM = layers.LSTM(self.nb_units, return_sequences=True, dropout=self.dropout, recurrent_dropout=self.dropout) # return_sequences=True
    
        super().build(input_shape)
        
    def call(self, x):
        x = self.reshape(x)
        x = self.LSTM(x)

        return x
    
class DecoderDense(tf.keras.layers.Layer):
    """
    Couches denses du décodeur intervenant après le LSTM du décodeur
    """
    def __init__(self, **kwargs):
        super(**kwargs).__init__()
    
    def build(self, input_shape):
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(36, activation='softmax')
    
        super().build(input_shape)
        
    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.flatten(x)

        return x


class Decoder(tf.keras.layers.Layer):
    """
    Ensemble des couches du décodeur, qui consiste en l'association des deux classes précédentes,
    càd des cellules LSTM puis des couches dense
    """
    def __init__(self, nb_units, dropout, **kwargs):
        self.nb_units = nb_units
        self.dropout = dropout
        super(**kwargs).__init__()
    
    def build(self, input_shape):
        self.rnn = DecoderRNN(nb_units=self.nb_units, dropout=self.dropout)
        self.dense = DecoderDense()
    
        super().build(input_shape)
        
    def call(self, x):
        x = self.rnn(x)
        x = self.dense(x)
        
        return x

def test():
    layer_input = tf.keras.Input(shape=(40, 188, 1))
    convLayers = ConvLayers()(layer_input)
    encoder_output = Encoder(nb_units=7*64, dropout=0.3)(convLayers)
    
    decoder_output = DecoderRNN(nb_units=64, dropout=0.3)(encoder_output)
    output = DecoderDense()(decoder_output)

    model = tf.keras.Model(layer_input, output)
    model.summary()
    
# test()

## **3) Entrainement du modèle**

In [ ]:
def get_model(path_save=""):
  """
  get_model initialise le modèle, ou le récupère s'il existe déjà

  Arguments
  ---------
  path_save: str
     Contient le chemin d'accès à un éventuel modèle déjà existant
     Si aucun modèle n'existe, il faut laisser path_save=""

  Returns
  -------
  model Sequential
  """

  if path_save != "":
    model = tf.keras.models.load_model(path_save)
    return model

  else:
    model = tf.keras.Sequential()
    model.add(ConvLayers())
    model.add(Encoder(nb_units=7*64, dropout=0.6))
    model.add(Decoder(nb_units=128, dropout=0.6))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy')

    return model


def predict_from_wav(model, path, n_mfcc=40):
    """
    predict_from_wav prend en entrée le modèle et le chemin d'un fichier audio, et effectue la prédiction
    """
    
    wave, sr = librosa.load(path, mono=True, sr=None)
    mfcc = wav2mfcc(wave, n_mfcc=n_mfcc)
    mfcc = np.asarray(mfcc)
    preds = model.predict(np.array([mfcc]))

    preds = preds.reshape(-1, 7, 36)
    preds = preds[0]
    plaque = ""
    for l in preds:
      entier = np.argmax(l)
      plaque += label_to_letter[entier]

    plaque = plaque[:2] + '-' + plaque[2:5] + '-' + plaque[5:]
    return plaque    

def plot(history, validation=False):
    """
    plot affiche l'évolution de la loss lors de l'entrainement
    Mettre validation=True s'il y a eu des données de validation lors de l'entrainement, sinon False
    """
    plt.plot(history.history['loss'])
    if (validation):
      plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    leg = ['train']
    if (validation):
      leg.append('test')
    plt.legend(leg, loc='upper left')
    plt.show()

In [ ]:
path_save = DRIVE_PATH + 'Model'
model = get_model(path_save)

Pour l'entrainement ci-dessous, on peut soit utiliser nos listes de MFCCs d'entrainement (qui sont fixes), soit utiliser un générateur afin de modifier les fichiers audio à chaque étape en y ajoutant du bruit. La deuxième méthode a pour inconvénient d'augmenter considérablement le temps d'entrainement (d'un facteur 50), à cause du temps de calcul des coefficients MFCCs.

In [ ]:
# Option 1 : Avec un générateur

from random import randint

def generator(waves, y, batch_size=64):
  compteur = 0
  n = len(waves)

  while True:
    waves, y = shuffle(waves, y, random_state=randint(0, 42))

    for step in range(0, n - batch_size, batch_size):
      compteur += 1
      X = modify_list_wav(waves[step: step+batch_size], ratio=30)
      X, _ = get_mfccs(X, y)

      yield X, y[step: step+batch_size]

data_generator = generator(waves, y, batch_size=128)

model.fit(data_generator, epochs=30)

In [ ]:
# Option 2 : Avec nos listes de MFCCs fixées

model.fit(X, y, epochs=30, batch_size=128, shuffle=True, plot_history=True)

In [ ]:
model.save(DRIVE_PATH + 'Model generator')

## **4) Prédictions**

In [ ]:
# Pour prédire à partir d'un fichier audio, on peut procéder de la manière suivante :

plaque = "am-881-rs"
prediction = model.predict_from_wav(path=DRIVE_PATH + "Audio test/" + plaque + ".wav")
print(prediction)

In [ ]:
def get_plaques(preds, y):
    """
    get_plaques prend en entrée les prédictions du modèle et les vrais labels, et renvoie les plaques d'immatriculation correspondantes
    -------------
    Arguments
    ---------
    preds: int list list
      Prédictions de la forme [[0,0,0,1,0,0,0,0,0,0,0,0...],
                               [0,0,0,0,0,1,0,0,0,0,0,0...]]
    
    y: int list list 
      Ce sont les vraies plaques d'immatriculation (de la même forme que preds), 
      et elles sont là afin de pouvoir les comparer avec les prédictions par la suite
      
    Output
    ------
    final_res: (str, str) list
      Liste de la forme [('ag-573-br', 'ag-583-br'),
                         ('bx-434-gf', 'bz-434-gf')],
    """    

    preds_translate = translate(preds)
    y_translate = translate(y)

    n = len(preds_translate)
            
    final_res = [(preds_translate[i], y_translate[i]) for i in range(n)]
    return final_res


def character_error_rate(preds, y, mode='all'):
  """
  character_error_rate prend en entrée les prédictions du modèle et les vrais labels, et renvoie le CER (character error rate)
  -------------
  Arguments
  ---------
  preds: int list list
    Prédictions de la forme [[0,0,0,1,0,0,0,0,0,0,0,0...],
                               [0,0,0,0,0,1,0,0,0,0,0,0...]]
    
  y: int list list 
    Les vraies plaques d'immatriculation (de la même forme que preds)
      
  mode: str
    Mode vaut 'all', 'number', ou 'letter'
    Il faut mettre 'all' si on veut le CER pour tous les caractères,
    ou 'number' si on le veut juste pour les nombres,
    ou 'letter' si on le veut juste pour les lettres

  Output
  ------
  print pourcentage (float)
  """    

  y_pred = translate(preds)
  y_true = translate(y)
  n = len(y_pred)
  m = len(y_pred[0]) - 2
  res = 0


  for i in range(n):
    new = 0

    if (mode !='letter' and mode != 'number'):
      for j in [0,1,3,4,5,7,8]:
        if y_pred[i][j] != y_true[i][j]:
          new += 1
      new /= m

    elif (mode == 'letter'):
      for j in [0,1,7,8]:
        if y_pred[i][j] != y_true[i][j]:
          new += 1
      new /= 4
      
    elif (mode == 'number'):
      for j in range(3,4,5):
        if y_pred[i][j] != y_true[i][j]:
          new += 1
      new /= 3
    
    res += new
  
  pourcentage = res/n*100
  pourcentage = round(pourcentage, 2)

  type_carac = "tous les caractères"
  if (mode=='number'):
    type_carac = "uniquement les nombres"
  elif (mode=='letter'):
    type_carac = "uniquement les lettres"

  print("{}% d'erreur sur {}".format(pourcentage, type_carac))

### **a) Prédictions sur la base d'entrainement**

In [ ]:
preds = model.predict(X)

print("Le CER (character error rate) est de : ")
character_error_rate(preds, y, mode='all')
print('--------------------------------')
character_error_rate(preds, y, mode='number')
character_error_rate(preds, y, mode='letter')
print('--------------------------------')

Le CER (character error rate) est de : 
0.61% d'erreur sur tous les caractères
--------------------------------
0.2% d'erreur sur uniquement les nombres
0.7% d'erreur sur uniquement les lettres
--------------------------------


In [ ]:
# Affichage des plaques prédites (à gauche) et des plaques réelles correspondantes
get_plaques(preds, y)

[('wj-990-yv', 'wj-990-yv'),
 ('md-410-jc', 'md-410-jc'),
 ('ck-862-eg', 'ck-862-eg'),
 ('gx-794-ll', 'gx-794-ll'),
 ('zm-693-oz', 'zm-693-oz'),
 ('pf-504-ha', 'pf-504-ha'),
 ('da-059-yi', 'da-059-yi'),
 ('kb-385-cw', 'kb-385-cw'),
 ('ep-482-au', 'ep-482-au'),
 ('ls-923-vl', 'ls-923-vl'),
 ('ug-931-od', 'ug-931-od'),
 ('px-972-fd', 'px-972-fd'),
 ('ud-922-jr', 'ud-922-jr'),
 ('cm-562-nq', 'cm-562-nq'),
 ('rb-380-lo', 'rb-380-lo'),
 ('ea-405-lx', 'ea-405-lx'),
 ('nu-825-np', 'nu-825-np'),
 ('fh-041-wj', 'fh-041-wj'),
 ('ai-064-ww', 'ai-064-ww'),
 ('ue-962-tk', 'ue-962-tk'),
 ('mi-561-hb', 'mi-561-hb'),
 ('ac-547-fl', 'ac-547-fl'),
 ('yz-961-kj', 'yz-961-kj'),
 ('ft-193-hh', 'ft-193-hh'),
 ('um-290-zo', 'um-290-zo'),
 ('qw-723-vm', 'qw-723-vm'),
 ('cb-112-cq', 'cb-112-cq'),
 ('eq-255-vk', 'eq-235-vk'),
 ('zw-941-os', 'zw-941-os'),
 ('wu-610-wk', 'wu-610-wk'),
 ('st-257-rg', 'st-257-rg'),
 ('rw-302-aq', 'rw-302-aq'),
 ('kg-861-ze', 'kg-861-ze'),
 ('ma-947-cw', 'ma-947-cw'),
 ('gf-230-hc',

### **b) Prédictions sur la base de test**

In [ ]:
preds_test = model.predict(X_test)

print("Le CER (character error rate) est de : ")
character_error_rate(preds_test, y_test, mode='all')
print('--------------------------------')
character_error_rate(preds_test, y_test, mode='number')
character_error_rate(preds_test, y_test, mode='letter')
print('--------------------------------')

Le CER (character error rate) est de : 
67.14% d'erreur sur tous les caractères
--------------------------------
14.0% d'erreur sur uniquement les nombres
76.5% d'erreur sur uniquement les lettres
--------------------------------


In [ ]:
# Affichage des plaques prédites (à gauche) et des plaques réelles correspondantes
get_plaques(preds_test, y_test)

[('vu-540-yv', 'wf-770-yf'),
 ('jn-335-wk', 'vl-116-wk'),
 ('ud-563-nl', 'uv-575-ey'),
 ('vy-890-jj', 'vz-800-jl'),
 ('mz-997-cl', 'yo-054-tj'),
 ('wb-763-eq', 'wp-669-dc'),
 ('pm-919-jv', 'ua-958-lm'),
 ('um-165-hj', 'ul-164-ga'),
 ('jn-752-dr', 'sl-682-js'),
 ('qb-764-wd', 'td-743-vd'),
 ('rr-757-oe', 'rx-517-pq'),
 ('kq-224-zb', 'qs-225-yb'),
 ('on-709-jy', 'pl-503-bx'),
 ('pd-253-mx', 'py-853-kq'),
 ('on-336-pa', 'pb-396-pa'),
 ('tm-611-hy', 'tn-683-fy'),
 ('rl-900-gn', 'rm-880-gn'),
 ('rj-501-oc', 'ra-491-pk'),
 ('ng-095-ww', 'nr-025-uw'),
 ('ly-427-pt', 'mz-427-ei'),
 ('yj-351-yq', 'nh-170-kq'),
 ('lh-185-ra', 'os-931-aa'),
 ('mj-142-zf', 'og-352-ju'),
 ('mm-429-jg', 'mm-420-gy'),
 ('jc-769-xg', 'gq-768-qr'),
 ('mg-928-pv', 'lz-180-tl'),
 ('ky-589-yy', 'kf-411-nz'),
 ('re-677-hf', 'jp-657-ho'),
 ('kc-220-oc', 'kx-220-fq'),
 ('pm-328-kv', 'il-308-kv'),
 ('sl-203-wn', 'hc-809-wn'),
 ('sq-539-db', 'hq-539-bl'),
 ('nq-832-jr', 'lq-032-ll'),
 ('hf-263-nj', 'ja-241-vh'),
 ('ny-679-ep',

### c) Estimation du CER obtenu par le hasard

In [ ]:
"""
Le code suivant effectue une estimation du CER obtenu par le hasard.
On génère un grand nombre de paires de plaques d'immatriculation, et on calcule le nombre d'erreurs en moyenne.

Notons qu'il aurait également été possible de calculer le CER obtenu par le hasard de manière mathématique.
"""

from random import randint

def estimation(mode='letter', n=50000):
  taille_voc = 26
  taille_echantillon = 4

  if (mode == 'number'):
    taille_voc = 10
    taille_echantillon = 3

  def gen():
    l = []
    for i in range(taille_echantillon):
      l.append(randint(0, taille_voc-1))

    return l

  def calcul(l1, l2):
    res = 0
    k = len(l1)
    for i in range(k):
      if l1[i] != l2[i]:
        res += 1
    return res/k

  res = 0
  n = 50000
  for i in range(n):
    l1 = gen()
    l2 = gen()
    res += calcul(l1, l2)

  print("Estimation du WER par le hasard pour {} : {}".format(mode, round(100*res/n, 1)))

estimation('letter')
estimation('number')

Estimation du WER par le hasard pour letter : 96.1
Estimation du WER par le hasard pour number : 90.1
